In [1]:
import torch

# Load the YOLOv5 model 
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  


Using cache found in C:\Users\bam/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-21 Python-3.10.11 torch-2.5.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
import cv2
import os

# Path to your images
image_folder = 'C:/Users/bam/Desktop/Week-7/notebooks/scraped_images/'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]

# Loop over all images for object detection
for img_path in image_files:
    img = cv2.imread(img_path)
    results = model(img)  
    results.show() 


In [ ]:
# Extract detection results
for result in results.xyxy[0]:  
    x1, y1, x2, y2, confidence, cls = result
    print(f"Class: {int(cls)}, Confidence: {confidence}, BBox: ({x1}, {y1}, {x2}, {y2})")


In [ ]:
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect("dbname=dbt_project.yml user=postgres password=time")
cur = conn.cursor()

# Insert detection data into the database
insert_query = """
INSERT INTO object_detection (image_path, class_label, confidence, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

for result in results.xyxy[0]:
    x1, y1, x2, y2, confidence, cls = result
    cur.execute(insert_query, (img_path, int(cls), confidence, x1, y1, x2, y2))

conn.commit()
cur.close()
conn.close()


In [ ]:
import logging

# Set up logging
logging.basicConfig(filename='object_detection.log', level=logging.INFO, 
                    format='%(asctime)s:%(levelname)s:%(message)s')

try:
    # Perform your detection
    results = model(img)
    logging.info(f"Successfully processed {img_path}")
except Exception as e:
    logging.error(f"Error processing {img_path}: {e}")
